In [55]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from tqdm import tqdm
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
import tensorflow as tf
import math
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from xgboost import plot_importance
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [3]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


### Remove songs which are in more than one playlist

In [4]:
dataset = []
data = np.array(df)
for i in df.index:
    l = eval(df["Playlists"][i])
    if len(l) > 1:
        continue
    data[i][-1] = str(l[0])
    dataset.append(data[i])
dataset = np.array(dataset)
        

In [5]:
print(len(df))
print(len(dataset))

11159
6120


In [6]:
counts = []
for i in range(169):
    counts.append(0)
for i in range(len(dataset)):
    counts[int(dataset[i][-1])] += 1
c = 0
playlists_to_keep = set()
for i in range(len(counts)):
    if counts[i] >= 100:
        c += 1
        playlists_to_keep.add(str(i))        
print(c)

13


In [7]:
print(type(dataset))

<class 'numpy.ndarray'>


In [8]:
temp_dataset = []
for c in dataset:
    if str(c[-1]) in playlists_to_keep:
        temp_dataset.append(c)
dataset = temp_dataset
dataset = np.array(dataset)
        

In [9]:
len(dataset)

2249

In [10]:
X = dataset[:, 1:-1]
y = dataset[:, -1]

In [58]:
def run_model(model_type = "LR", neighbours = 5):
    playlists = []

    np.random.seed(123)

    X = dataset[:, 1:-1]
    y = dataset[:, -1]
    
    s = set()
    for c in y:
        s.add(c)
    
    if model_type == 'ANN':
        le = preprocessing.LabelEncoder()
        le.fit(y)
        y = le.transform(y)
        
        enc = OneHotEncoder()
        y = y.reshape(-1, 1)
        enc.fit(y)
        y = enc.transform(y).toarray()
        
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify = y)

    standardScalar = StandardScaler()
    X_train = standardScalar.fit_transform(X_train)
    X_test = standardScalar.transform(X_test)
     
    
    if(model_type == "LR" ):
        model = LogisticRegression()

    elif(model_type == "SVM"):
        model = SVC(kernel = 'linear')

    elif (model_type == "DT"):
        model = DecisionTreeClassifier(criterion = 'entropy', max_depth =  20)

    elif (model_type == "RF"):
        model = RandomForestClassifier(criterion = 'entropy')

    elif(model_type == "XGB"):

        param = {
            "learning_rate" : 0.1,
            "n_estimators" : 500,
            "max_depth" : 20,
            "min_child_weight" : 1,
            "gamma" : 0.1,
            "subsample": 0.9,
            "colsample_bytree" : 0.9,
            "objective" : 'binary:logistic',
            "nthread" : 4,
            "scale_pos_weight" :  1,
            "seed" : 27
        }

        model = XGBClassifier(**param)

    elif(model_type == "KNN"):
        model = KNeighborsClassifier(n_neighbors=neighbours)

    elif(model_type == "ANN"):
        model = Sequential([Dense(units = 64, input_shape = (34, ), activation = 'relu'),
        Dense(units = 32, activation = 'relu'),
        Dense(units = 16, activation = 'relu'),       
        Dense(units = len(s), activation = 'softmax')
        ])
        model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.05), loss = 'categorical_crossentropy', metrics = ['accuracy'])
        model.summary()




#         xgb = XGBClassifier(**param)    

    if model_type != 'ANN':
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
#         if model_type == 'XGB':
#             plot_importance(model)
#             plt.show()
#         if model_type == 'SVM':
#             f_importances(model.coef_, df.columns[1:-1])
        if model_type == 'RF':
            importances = model.feature_importances_
            combined_list = []
            features = list(df.columns[1:-1])
            
            s = 0
            for i in range(len(importances)):
                if "lda" in features[i]:
                    s += importances[i]
                    continue
                combined_list.append((importances[i], features[i]))
            combined_list.append((s, "all topics combined"))
            combined_list.sort()
            print(combined_list)
            features = []
            importances = []
            for c in combined_list:
                features.append(c[1])
                importances.append(c[0])
                
            indices = np.argsort(importances)

            plt.title('Feature Importances')
            plt.barh(range(len(features)), importances, color='b', align='center')
            plt.yticks(range(len(features)), features)
            plt.xlabel('Relative Importance')
            plt.show()

    else:
        model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 64, epochs = 100)
        

        y_pred = model.predict_classes(X_test)
#         print(y_test)
        temp = []
        for c in y_test:
            for i in range(len(c)):
                if c[i] == 1:
                    temp.append(i+1)
        y_test = temp
        y_test = np.array(y_test)
    
#     print(y_pred)
#     print(y_test)

    pres_score = precision_score(y_test, y_pred, average = "macro")
    acc_score = accuracy_score(y_test, y_pred)
    
    
    f1= f1_score(y_test, y_pred, average = "macro")
    recall_Score = recall_score(y_test, y_pred, average = "macro")

    result_history = {"Precision Score " : pres_score, "Recall Score " : recall_Score, "F1 Score " : f1, "Accuracy Score ": acc_score}


    print(result_history)
    return result_history

In [41]:
result_history = run_model(model_type = "XGB")

[22:23:03] WARNING: ../src/learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


{'Precision Score ': 0.5926373839465681, 'Recall Score ': 0.5819345582431832, 'F1 Score ': 0.585059532938555, 'Accuracy Score ': 0.6022222222222222}


In [59]:
result_history = run_model(model_type = "RF")

[(0.007014089902247543, 'mode'), (0.01855788829851921, 'key'), (0.025308209412053605, 'liveness'), (0.028068341672251612, 'tempo'), (0.035752862209776304, 'duration_ms'), (0.04654799667442491, 'instrumentalness'), (0.05726222912868504, 'valence'), (0.0609829472084903, 'speechiness'), (0.06279872763674345, 'loudness'), (0.06668418186019734, 'popularity'), (0.0671916854341815, 'danceability'), (0.07268189011878806, 'energy'), (0.08980457367525346, 'acousticness'), (0.09871041883165428, 'release_date'), (0.2626339579367333, 'all topics combined')]


ValueError: RandomForestClassifier should be a binary classifier

In [ ]:
result_history = run_model(model_type = "KNN", neighbours = 23)

In [ ]:
result_history = run_model(model_type = "LR")

In [24]:
result_history = run_model(model_type = "SVM")

['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'release_date', 'popularity', 'lda_topic_0', 'lda_topic_1', 'lda_topic_2', 'lda_topic_3', 'lda_topic_4', 'lda_topic_5', 'lda_topic_6', 'lda_topic_7', 'lda_topic_8', 'lda_topic_9', 'lda_topic_10', 'lda_topic_11', 'lda_topic_12', 'lda_topic_13', 'lda_topic_14', 'lda_topic_15', 'lda_topic_16', 'lda_topic_17', 'lda_topic_18', 'lda_topic_19']


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
result_history = run_model(model_type = "DT")

In [ ]:
result_history = run_model(model_type = "ANN")

In [ ]:
# f = open("binary_svc_linear_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
# f = open("binary_svc_linear_result", "rb")
# l = pickle.load(f)
# f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])